Project:  Identify malarial parasite in blood cells in resource poor settings from microscopic pictures taken by cellphone. 

         Create model on a regular laptop without GPU
         
         Deploy the model in a phone on on the web
         
         
inspiration: https://blog.insightdatascience.com/https-blog-insightdatascience-com-malaria-hero-a47d3d5fc4bb by Carlos Atico Ariza

This version uses Squeezenet


In [1]:

# importing turicreate as tc

import warnings
# ignoring warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import turicreate as tc

Data source: Data was obtained from https://ceb.nlm.nih.gov/repositories/malaria-datasets/

There is a link to download the files at the bottom of the website. 

Download and unzip the files to your working directory for this Jupyter notebook

In [2]:
# Loading the images from local drive.

reference_data  = tc.image_analysis.load_images('cell_images', with_path=True)

Read 5960 images in 5.00091 secs	speed: 1191.78 file/sec

Read 10488 images in 10.0018 secs	speed: 905.438 file/sec

Read 15371 images in 15.0039 secs	speed: 976.2 file/sec

Read 20422 images in 20.0055 secs	speed: 1009.88 file/sec

In [3]:
# checking the data 
reference_data.head()

path,image
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 148 Width: 142
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 208 Width: 148
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 151 Width: 139
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 154 Width: 145
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 154 Width: 133
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 145 Width: 169
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 139 Width: 139
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 124 Width: 154
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 148 Width: 124
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 142 Width: 139


In [4]:
image_sarray = reference_data['image']

In [5]:
# Resizing the image -you can skip this 

resized_images = tc.image_analysis.resize(image_sarray,224,224)

In [6]:
# Reinstating the edited "image" column back to reference_data

reference_data['image'] = resized_images

In [7]:
# Labelling each images in the SFrame using the folder name as malaria or not malaria 

reference_data['label'] = reference_data['path'].apply(lambda path: 'malaria' if '/Parasitized' in path else 'not malaria')


In [8]:
# Displaying first 5 rows to make sure that we got the data in the right format

reference_data.head()

path,image,label
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria
cell_images/Parasitized/C100P61ThinF_IMG_20150 ...,Height: 224 Width: 224,malaria


In [9]:
#Splitting the data into training, testing and validation sets

train_data2, test_data = reference_data.random_split(0.8)
train_data, valid_data = train_data2.random_split(0.8)

In [ ]:
# Create model
model = tc.image_classifier.create(train_data, target= 'label', max_iterations = 100,validation_set = valid_data, model='squeezenet_v1.1'  )

In [24]:
# storing the prediction 

predictions = model.predict(test_data)

In [ ]:
# Evaluating the model on the test data

metrics = model.evaluate(test_data)
print(metrics['accuracy'])

In [26]:
# Printing confusion matrix

print ("Confusion Matrix : \n%s" % metrics['confusion_matrix'])

Confusion Matrix : 
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
|   malaria    |     malaria     |  2584 |
|   malaria    |   not malaria   |  211  |
| not malaria  |   not malaria   |  2722 |
| not malaria  |     malaria     |  114  |
+--------------+-----------------+-------+
[4 rows x 3 columns]



In [27]:
model.summary()

Class                                    : ImageClassifier

Schema
------
Number of classes                        : 2
Number of feature columns                : 1
Input image shape                        : (3, 227, 227)

Training summary
----------------
Number of examples                       : 17443
Training loss                            : 2665.2744
Training time (sec)                      : 790.3401



In [28]:
#It took 13 minutes to complete model generation from 22,147 images on a laptop with 
# Intel core i7-6500U 2.5GHZ , 16 gb memory without using the GPU

In [29]:
 #Save the model for later use in Turi Create

model.save('malaria_squeeze100.model')

In [30]:
#Export for use in Core ML
# You can use this model in the template provided by apple to create an iphone app
# Refer to this website if you need assistance - https://www.appcoda.com/core-ml-model-with-python/

model.export_coreml('malaria_squeeze100.mlmodel')

In [31]:
print(metrics['f1_score'])

0.943664413243


In [32]:
targets = test_data['label']

In [33]:
f1 = tc.evaluation.f1_score(targets, predictions)

In [34]:
fbeta = tc.evaluation.fbeta_score(targets, predictions, beta = 2.0)

In [35]:
print(f1, fbeta)

(0.9436644132431965, 0.9532815017160468)


Stats

F-beta score = 95.3

Accuracy = 94.23%

Sensitivity = 92.45%

Specificity = 95.98 %

Positive predictive value = 95.77%

Negative predictive value = 92.81 %

False Discovery Rate =	4.23%

Matthews Correlation Coefficient =	88.51%

Diagnostic odds ratio = 292.38


In [36]:
print(metrics)

{'f1_score': 0.9436644132431965, 'auc': 0.9817053170203646, 'recall': 0.9598025387870239, 'precision': 0.9280600068189567, 'log_loss': 0.16909834122102477, 'roc_curve': Columns:
	threshold	float
	fpr	float
	tpr	float
	p	int
	n	int

Rows: 100001

Data:
+-----------+----------------+----------------+------+------+
| threshold |      fpr       |      tpr       |  p   |  n   |
+-----------+----------------+----------------+------+------+
|    0.0    |      1.0       |      1.0       | 2836 | 2795 |
|   1e-05   | 0.849373881932 |      1.0       | 2836 | 2795 |
|   2e-05   | 0.826118067979 |      1.0       | 2836 | 2795 |
|   3e-05   | 0.810733452594 |      1.0       | 2836 | 2795 |
|   4e-05   | 0.793559928444 | 0.999647390691 | 2836 | 2795 |
|   5e-05   | 0.780322003578 | 0.999647390691 | 2836 | 2795 |
|   6e-05   | 0.772808586762 | 0.999647390691 | 2836 | 2795 |
|   7e-05   | 0.763864042934 | 0.999647390691 | 2836 | 2795 |
|   8e-05   | 0.757781753131 | 0.999647390691 | 2836 | 2795 |
|   